<a href="https://colab.research.google.com/github/blackpearl-09/TIL/blob/main/cosine_similarity_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## - The process of data analysis for text data

 1. 형태소 분석(Tokenize)
 2. 품사 표시(POS Tagging, Part-of-speech)
 3. 불용어(Stopwords) 제거 
 --------------------------------------
 4. 단어 개수 카운팅 및 단어 사전 생성
 5. 단어 사전 기반 데이터 시각화 
 6. 머신러닝 / 딥러닝 모델 적용
 

### 자연어처리를 위한 KoNLPy패키지 


 - 클래스: Hannanum, Kkma, Komoran, Okt(Twitter) 등이 있다.
 
 
 - 공통 내장함수 
  1. morphs(text) : 텍스트에서 형태소를 반환한다
  2. nouns(text) : 텍스트에서 명사를 반환한다
  3. phrases(text) : 텍스트에서 어절을 뽑아낸다
  4. pos(text) : 텍스트에서 품사 정보를 부착하여 반환한다

### Okt (Open Korean Text), 형태소 분석기
- 오픈 소스 한국어 분석기. 트위터에서 만든 오픈소스 한국어 처리기인 twitter-korean-text를 이어받음


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt 
from numpy import dot 
from numpy.linalg import norm #벡터의 크기 측정하는 함수
import numpy as np 
import pandas as pd
okt = Okt()


text1 = '안녕 나는 맨유로 이적한 호날두야' 
text2 = '안녕 나는 맨시티로 이적하지 못하고 토트넘에 잔류한 케인이야' 
text3 = '나는 맨시티와 케인을 좋아해. 호날두는 별로야'


print(okt.nouns(text1))
print(okt.nouns(text2))
print(okt.nouns(text3))
print(type(okt.nouns(text1)))

['안녕', '나', '맨유', '이적', '호날두']
['안녕', '나', '맨시티', '이적', '못', '토트넘', '잔류', '케인']
['나', '맨시티', '케인', '호날두', '별로']
<class 'list'>


In [ ]:
texts = [text1, text2, text3]
vector = CountVectorizer() #countvetorizer 생성 
corpus =[]

for text in texts:  #회사명을 하나씩 value에 넣고
    corpus.append(text)

text_arr = vector.fit_transform(corpus).toarray()


print(vector.vocabulary_) # 각 단엉의 배열 순서 

pd.DataFrame(text_arr).head()#순서에 따라 text1~3 벡터화

{'안녕': 6, '나는': 0, '맨유로': 3, '이적한': 8, '호날두야': 15, '맨시티로': 1, '이적하지': 7, '못하고': 4, '토트넘에': 13, '잔류한': 9, '케인이야': 12, '맨시티와': 2, '케인을': 11, '좋아해': 10, '호날두는': 14, '별로야': 5}


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1
1,1,1,0,0,1,0,1,1,0,1,0,0,1,1,0,0
2,1,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0


#### 주의할 것은 CountVectorizer는 오직 띄어쓰기만을 기준으로 단어를 자른다는 점
 - 영어의 경우 띄어쓰기만으로 토큰화가 수행되기 때문에 문제가 없지만 한글에 CountVectorizer를 적용하면, 조사 등의 이유로 제대로 BoW가 만들어지지 않음. 

In [ ]:
def make_matrix(feats, list_data): 
    freq_list = [] 
    for feat in feats: 
        freq = 0 
        for word in list_data: 
            if feat == word: 
                freq += 1 
        freq_list.append(freq) 
    return freq_list

In [ ]:
def cos_sim(A,B): 
    np.seterr(divide = 'ignore', invalid = 'ignore')
    return dot(A, B)/(norm(A)*norm(B))


In [ ]:
v1 = okt.nouns(text1) 
v2 = okt.nouns(text2)
v3 = okt.nouns(text3) 
# 단어들을 중복제거를 위해, set에 데이터를 쌓는다

v4 = v1 + v2 + v3 
t_set= set(v4) 
print(t_set)

v1_arr = np.array(make_matrix(t_set, v1)) 
v2_arr = np.array(make_matrix(t_set, v2)) 
v3_arr = np.array(make_matrix(t_set, v3)) 

cs1 = cos_sim(v1_arr, v2_arr) 
cs2 = cos_sim(v1_arr, v3_arr) 
cs3 = cos_sim(v2_arr, v3_arr) 
print('v1 <-> v2 = ', cs1) 
print('v1 <-> v3 = ', cs2) 
print('v2 <-> v3 = ', cs3)



{'안녕', '못', '이적', '맨유', '호날두', '잔류', '케인', '토트넘', '별로', '나', '맨시티'}
v1 <-> v2 =  0.4743416490252569
v1 <-> v3 =  0.3999999999999999
v2 <-> v3 =  0.4743416490252569


In [ ]:
# texts = [text1, text2, text3]

# vector = CountVectorizer() #countvetorizer 생성 
# corpus =[]

# for text in texts:  #회사명을 하나씩 value에 넣고
#     text = okt.nouns(text)
#     corpus.append(text)

# text_arr = vector.fit_transform(corpus).toarray()

# print(text_arr) #순서에 따라 text1~3 벡터화
# print(vector.vocabulary_) # 각 단엉의 배열 순서 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline


vectorizer = CountVectorizer(analyzer = 'word', 
                         tokenizer = None,
                         preprocessor = None, 
                         stop_words = None, 
                         min_df = 2, # 토큰이 나타날 최소 문서 개수
                         ngram_range=(1, 3),
                         max_features = 20000
                        )
vectorizer

* stop_words : 리스트 형태로 불용어로 처리하고자 하는 문자를 넣어주시면 됩니다.

* analyzer : 문자열 {‘word’, ‘char’, ‘char_wb’} 또는 함수로 어떤 단위로 토큰화 할지를 정의합니다.

* token_pattern : 토큰화 하는 정규표현식 패턴입니다.

* tokenizer : 사이킷런에서 기본으로 제공하고 있는 토큰화 방법이 아닌 커스텀한 함수로 토큰화 하고자 할 때 함수를 만들어서 넣어줄 수 있습니다.

* ngram_range : n-그램 범위로 단어를 몇 개로 토큰화 할지를 의미합니다.

* max_df : 문서에서 등장하는 최대 빈도수를 의미합니다.

* min_df : 문서에서 등장하는 최소 빈도수를 의미합니다.